In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage

from keras import models
from keras import layers
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras import optimizers
from keras_preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import DenseNet121
from keras.applications.xception import Xception
from keras.applications import VGG16
from keras.applications import NASNetLarge, InceptionV3
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, MaxPool2D, Flatten, Dropout, CuDNNLSTM, CuDNNGRU, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.optimizers import Adam, SGD, Nadam
from keras.layers.normalization import BatchNormalization
from time import time
from tensorflow.python.client import device_lib

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from keras.preprocessing import image
import os, shutil
from IPython.display import display
import os
import cv2
import math
import json

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from hyperas.distributions import uniform
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

In [ ]:
bread_labels = pd.read_csv('../pet_data/breed_labels.csv')
color_labels = pd.read_csv('../pet_data/color_labels.csv')
state_labels = pd.read_csv('../pet_data/state_labels.csv')
test_sample_submission = pd.read_csv('../pet_data/test/sample_submission.csv')
test_df = pd.read_csv('../pet_data/test/test.csv')
train_df = pd.read_csv('../pet_data/train.csv')
train_folder = '../pet_data/train_images/'
test_folder = '../pet_data/test_images/'

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
name_target_dict = train_df.set_index('PetID')['AdoptionSpeed'].to_dict()
name_target_dict_type = train_df.set_index('PetID')['Type'].to_dict()
train_image_names = os.listdir('../pet_data/train_images')
test_image_names = os.listdir('../pet_data/test_images')

In [ ]:
def display_samples(columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 3*rows))

    for i in range(columns*rows):
        image_path = train_image_names[i]
        image_id = name_target_dict[image_path.split('-')[0]]
        img = cv2.imread(f'../pet_data/train_images/{image_path}')
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)

display_samples()

In [ ]:
generator_dict = {'filename': [], 'class': [], 'type': []}

for name in train_image_names:
    short_name = name.split('-')[0]
    label = name_target_dict[short_name]
    t = name_target_dict_type[short_name]
    
    generator_dict['filename'].append(name)
    generator_dict['class'].append(label)
    generator_dict['type'].append(str(t))
generator_df = pd.DataFrame(generator_dict)
generator_df.head(8)

In [ ]:
generator_df.groupby('type').size().plot.bar()

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title='confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[: np.newaxis]
        print('normalized confusion matrix')
    else:
        print('unnormalized confusion matrix')
        
    print(cm)
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment='center',
                color='white' if cm[i, j] > thresh else 'black'
                )
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


def plot_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('model accuracy')
    plt.xlabel('accuracy')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('model loss')
    plt.xlabel('loss')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


In [ ]:
import matplotlib.image as mpimg
def show_image_predictions(probabilities, direct):
    for index, probability in enumerate(probabilities):
        image_path = direct + "/" +test_generator.filenames[index]
        img = mpimg.imread(image_path)
    #     with open(TEST_FILE,"a") as fh:
    #         fh.write(str(probability[0]) + " for: " + image_path + "\n")
        plt.imshow(img)
        if probability > 0.5:
            plt.title("%.2f" % (probability[0]*100) + "% dog in " + image_path)
        else:
            plt.title("%.2f" % ((1-probability[0])*100) + "% cat")
        plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
random_state = 42
train, test = train_test_split(generator_df, test_size=0.2, random_state=random_state)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


def plot_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('model accuracy')
    plt.xlabel('accuracy')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('model loss')
    plt.xlabel('loss')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

# Augmentation

In [ ]:
plot_history(history)

In [ ]:
# model.load_weights('model.h5')

val_scores = model.evaluate_generator(
    valid_generator,
    STEP_SIZE_VALID,
    verbose=1,
)

print('Validation loss:', val_scores[0])
print('Validation accuracy:', val_scores[1])

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
# cut off the last 3 rows as it messes 
# with the predictions due to the batch size not dividing evenly
test_mod = test[:-3]
test_generator = test_datagen.flow_from_dataframe(
        test_mod, 
        '../pet_data/train_images/', 
        x_col='filename',
        y_col=None, 
        target_size=(150, 150),
        class_mode=None, 
        color_mode='rgb',
        batch_size=16, 
        shuffle=False,
        seed=2018,
    )

STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [ ]:
test_generator.reset()
pred=model.predict_generator(
    test_generator,
    STEP_SIZE_TEST,
    verbose=1
)
print('\nTest loss:', pred[0])
print('Test accuracy:', pred[1])

In [ ]:
prediction_dict = {'filename': [], 'prediction': []}
df = pd.DataFrame()

for index, predict in enumerate(pred):
    filename = test_generator.filenames[index]
    if predict > 0.5:
        prediction_dict['filename'].append(filename) 
        prediction_dict['prediction'].append(2) 
    else: 
        prediction_dict['filename'].append(filename) 
        prediction_dict['prediction'].append(1)

predictions_df = pd.DataFrame(prediction_dict) 

y_true = test_mod.type.astype(int)
y_pred = predictions_df.prediction

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_true, y_pred),
    ['cat', 'dog']
)

In [ ]:
report = classification_report(
    y_true, 
    y_pred, 
    [1, 2]
)
print(report) 

In [ ]:
for index, predict in enumerate(pred):
    image_path = '../pet_data/train_images/' +test_generator.filenames[index]
    img = mpimg.imread(image_path)
    plt.imshow(img)
    if predict > 0.5:
        plt.title("%.2f" % (predict[0]*100) + "% cat")
    else:
        plt.title("%.2f" % ((1-predict[0])*100) + "% dog")
    plt.show()

# Adding Hyperas

In [2]:
def data():
    test_df = pd.read_csv('../pet_data/test/test.csv')
    train_df = pd.read_csv('../pet_data/train.csv')
    train_folder = '../pet_data/train_images/'
    test_folder = '../pet_data/test_images/'
    name_target_dict = train_df.set_index('PetID')['AdoptionSpeed'].to_dict()
    name_target_dict_type = train_df.set_index('PetID')['Type'].to_dict()
    train_image_names = os.listdir('../pet_data/train_images')
    test_image_names = os.listdir('../pet_data/test_images')
    generator_dict = {'filename': [], 'class': [], 'type': []}

    for name in train_image_names:
        short_name = name.split('-')[0]
        label = name_target_dict[short_name]
        t = name_target_dict_type[short_name]

        generator_dict['filename'].append(name)
        generator_dict['class'].append(label)
        generator_dict['type'].append(str(t))
        
    generator_df = pd.DataFrame(generator_dict)
    generator_df.head(8)
    
    random_state = 42
    train, test = train_test_split(generator_df, test_size=0.2, random_state=random_state)
    
    train_datagen=ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.1,  # set range for random shear
        zoom_range=0.1,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=1/255.,
        # set function that will be applied on each input
        preprocessing_function=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.1
    )
    
    train_generator = train_datagen.flow_from_dataframe(
        train, 
        '../pet_data/train_images/', 
        x_col='filename',
        y_col='type', 
        has_ext=True,  # If image extension is given in x_col
        target_size=(150, 150), 
        color_mode='rgb',
        class_mode='binary', 
        batch_size=16, 
        shuffle=True, 
        seed=2018,
        subset='training'
    )
    valid_generator = train_datagen.flow_from_dataframe(
        train, 
        '../pet_data/train_images/', 
        x_col='filename',
        y_col='type', 
        has_ext=True,  # If image extension is given in x_col
        target_size=(150, 150), 
        color_mode='rgb',
        class_mode='binary', 
        batch_size=16, 
        shuffle=True, 
        seed=2018,
        subset='validation'
    )
    
    return train_generator, valid_generator, train, test

In [3]:
def model(train_generator, valid_generator):
    model = models.Sequential()
    
    ks1_first = {{choice([2, 3, 4, 5, 8, 10])}}
    ks1_second = {{choice([2, 3, 4, 5, 8, 10])}}
    
    ks2_first = {{choice([2, 3, 4, 5, 8, 10])}}
    ks2_second = {{choice([2, 3, 4, 5, 8, 10])}}
    
    input_shape=(150, 150, 3)
    
    model.add(Conv2D(filters=( {{choice([1, 2, 3, 4, 5, 8, 12])}} ), 
                     kernel_size=(ks1_first, ks1_second),
                     input_shape=input_shape, 
                     padding='same',
                     kernel_initializer='TruncatedNormal',
                     activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    for _ in range( {{choice([0, 1, 2, 3])}} ):
        model.add(Conv2D(filters=( {{choice([4, 8, 12])}} ), 
                     kernel_size= (ks2_first, ks2_second), 
                         padding='same',
                     kernel_initializer='TruncatedNormal',
                         activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D((2, 2))) 
    
    model.add(layers.Flatten())
    
    for _ in range( {{choice([0, 1, 2, 3, 4, 8, 16, 32])}} ):
        model.add(Dropout( {{uniform(0, 1)}} ))
        model.add(Dense( {{choice([4, 8, 16, 32, 64, 128, 256, 512])}} , 
                        kernel_initializer='TruncatedNormal',
                       activation='relu'))
        model.add(layers.BatchNormalization())
       
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer={{choice([
                      optimizers.RMSprop(lr=1e-4),
                      'Adam',
                      'adadelta',
                      'adagrad',
                      'nadam'
                  ])}},
                  metrics=['acc'])
    
    early_stopping_monitor = EarlyStopping(patience=100)
    
#     schedule = SGDRScheduler(min_lr= {{uniform(1e-8, 1e-5)}} ,
#                                      max_lr= {{uniform(1e-3, 1e-1)}} ,
#                                      steps_per_epoch=np.ceil(epoch_size/bs),
#                                      lr_decay=0.9,
#                                      cycle_length=5, # 5
#                                      mult_factor=1.5)
    checkpoint = ModelCheckpoint(
        'model_with_hyperas.h5', 
        monitor='val_acc', 
        verbose=0, 
        save_best_only=True, 
        save_weights_only=False,
        mode='auto'
    )

    STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

    result = model.fit_generator(
        train_generator,
        steps_per_epoch=STEP_SIZE_TRAIN, 
        epochs=1,
        validation_data=valid_generator,
        validation_steps=STEP_SIZE_VALID,
        callbacks=[early_stopping_monitor, checkpoint],
        verbose=0
    )
    
    val_acc = np.amin(result.history['val_acc'])
    print('Best validation accuracy of epoch:', val_acc)
    
    return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

In [4]:
best_run, best_model = optim.minimize(
    model=model, 
    data=data,
    algo=tpe.suggest,
    max_evals=5,
    eval_space=True,
    trials=Trials(),
    notebook_name='Train cat dog classifier hyperparam tuning with augmentation'
)
    
X_train, y_train, X_test, y_test = data()
print("Evalutation of best performing model:")
# print(best_model.evaluate(valid))

print("Crossvalidation of best performing model:")
#print(best_model.evaluate(X_test, y_test))

print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from scipy import ndimage
except:
    pass

try:
    from keras import models
except:
    pass

try:
    from keras import layers
except:
    pass

try:
    from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
except:
    pass

try:
    from keras.layers import Dense, Dropout, GlobalAveragePooling2D
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    from keras_preprocessing.image import ImageDataGenerator
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint
except:
    pass

try:
    from tensorflow.keras.applications import DenseNet121
except:
    pass

try:
    from keras.applications.xception import Xception
ex

  79: 
Found 7438 images belonging to 2 classes.
Found 826 images belonging to 2 classes.
Instructions for updating:
Colocations handled automatically by placer.
  0%|          | 0/5 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From C:\Users\darre\Anaconda3\envs\project_new\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Best validation accuracy of epoch:                 
0.5281862745098039                                 
Best validation accuracy of epoch:                                            
0.491358024764944                                                             
Best validation accuracy of epoch:                                            
0.528395061801981                  